# Cria o mapa com a área de influência regional por rota (origem ou destino)

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.graph_objects as go
from geopy.distance import geodesic
from shapely.geometry import Polygon
from shapely.geometry import Point
from geovoronoi import voronoi_regions_from_coords

In [3]:
id_aeroporto = pd.read_excel("data/excel/xlsx/id_aeroporto_v2.xlsx", engine='openpyxl')
municipios = gpd.read_file("data/shapefile/simplificado/municipios_simplificado.shp")
zona_de_influencia = pd.read_excel("data/excel/xlsx/zona_de_influencia.xlsx", engine='openpyxl')

In [4]:
def gerar_circulo(coordenadas_principais, raio_km, num_circle_points=360):
    """Gera um polígono circular em torno de um ponto central."""
    circle_points = []
    for i in range(num_circle_points):
        angle = 360 * i / num_circle_points
        circle_point = geodesic(kilometers=raio_km).destination(coordenadas_principais, angle)
        circle_points.append((circle_point.longitude, circle_point.latitude))
    return Polygon(circle_points)

def filtrar_shapefile_por_circulo(shapefile, circle_polygon):
    """Filtra o shapefile para incluir apenas os municípios que intersectam o círculo."""
    # Filtra os municípios que intersectam o círculo
    shapefile = shapefile[shapefile.intersects(circle_polygon)].copy()

    # Modifica a coluna 'geometry' com base na interseção
    shapefile.loc[:, 'geometry'] = shapefile['geometry'].intersection(circle_polygon)

    # Remove geometrias vazias
    shapefile = shapefile[~shapefile['geometry'].is_empty]

    return shapefile

def calcular_zoom(raio_km):
    fator_ajuste = 12
    return fator_ajuste - np.log(raio_km)

def gerar_mapa_aeroporto(df, id_aeroporto_nome, id_aeroporto_filtros, shapefile=None, zona_de_influencia=None):
    # Identificar o aeroporto principal
    aeroporto = df.loc[df['ID_AEROPORTO'] == id_aeroporto_nome].iloc[0]
    latitude = aeroporto['LATITUDE'].astype('float32')
    longitude = aeroporto['LONGITUDE'].astype('float32')
    coordenadas_principais = (latitude, longitude)
    uf_aeroporto = aeroporto['UF']
    classif_aeroporto = aeroporto['CLASSIF']
    
    # Obter o valor de distancia_maxima e outras métricas para o UF do aeroporto principal
    if zona_de_influencia is not None:
        raio_km = float(zona_de_influencia.loc[zona_de_influencia['UF'] == uf_aeroporto, 'distancia_maxima'].values[0])
        mediana_km = float(zona_de_influencia.loc[zona_de_influencia['UF'] == uf_aeroporto, 'mediana'].values[0])
        decil_6_km = float(zona_de_influencia.loc[zona_de_influencia['UF'] == uf_aeroporto, 'decil_6'].values[0])
        decil_7_km = float(zona_de_influencia.loc[zona_de_influencia['UF'] == uf_aeroporto, 'decil_7'].values[0])
        decil_8_km = float(zona_de_influencia.loc[zona_de_influencia['UF'] == uf_aeroporto, 'decil_8'].values[0])
        decil_9_km = float(zona_de_influencia.loc[zona_de_influencia['UF'] == uf_aeroporto, 'decil_9'].values[0])
    else:
        raise ValueError("Zona de influência não fornecida ou UF não encontrado na zona de influência.")

    # Criar DataFrame para armazenar os dados de influência dos municípios
    influencia_municipios_data = []
    
    # Filtrar aeroportos similares com base na classificação e número de passageiros > 0, com exceção para UF == 'PI'
    if classif_aeroporto == "Classe IV":
        aeroportos_similares = df.loc[
            (df['CLASSIF'].isin(["Classe IV", "Classe III"])) & 
            ((df['PASSAGEIROS_ULTIMO_ANO'] > 0))
        ].copy()
    elif classif_aeroporto == "Classe III":
        aeroportos_similares = df.loc[
            (df['CLASSIF'].isin(["Classe IV", "Classe III", "Classe II"])) & 
            ((df['PASSAGEIROS_ULTIMO_ANO'] > 0))
        ].copy()
    else:
        aeroportos_similares = df.loc[
            (df['CLASSIF'].isin(["Classe IV", "Classe III", "Classe II", "Classe I"])) & 
            ((df['PASSAGEIROS_ULTIMO_ANO'] > 0))
        ].copy()

    # Verificar se "ALL" está em id_aeroporto_filtros e, nesse caso, não aplicar filtros adicionais
    if "ALL" in id_aeroporto_filtros:
        aeroportos_filtrados = aeroportos_similares
    else:
        # Filtrar aeroportos com base em ORIGENS e DESTINOS que contenham algum dos ID de id_aeroporto_filtros
        aeroportos_filtrados = aeroportos_similares.loc[
            aeroportos_similares.apply(lambda row: any(filtro in row['ORIGENS'] for filtro in id_aeroporto_filtros) or 
                                                any(filtro in row['DESTINOS'] for filtro in id_aeroporto_filtros), axis=1)
        ].copy()

    # Filtrar aeroportos dentro do círculo, excluindo o aeroporto principal
    aeroportos_proximos = aeroportos_similares.loc[
        (df['ID_AEROPORTO'] != id_aeroporto_nome) & 
        (df.apply(lambda row: geodesic(coordenadas_principais, (row['LATITUDE'], row['LONGITUDE'])).km <= raio_km, axis=1))
    ].copy()

    # Verificar se "ALL" está em id_aeroporto_filtros e, nesse caso, não aplicar filtros adicionais
    if "ALL" in id_aeroporto_filtros:
        aeroportos_proximos_filtrados = aeroportos_proximos
    else:
        # Filtrar aeroportos com base em ORIGENS e DESTINOS que contenham algum dos ID de id_aeroporto_filtros
        aeroportos_proximos_filtrados = aeroportos_proximos.loc[
            aeroportos_proximos.apply(lambda row: any(filtro in row['ORIGENS'] for filtro in id_aeroporto_filtros) or 
                                                any(filtro in row['DESTINOS'] for filtro in id_aeroporto_filtros), axis=1)
        ].copy()    

    # Definir os limites do quadrado
    lat_min, lat_max = -40, 10  # Extensão das latitudes (Sul e Norte)
    lon_min, lon_max = -80, -20  # Extensão das longitudes (Oeste)

    # Criar o quadrado que contém todo o Brasil
    square_brasil = Polygon([
        (lon_min, lat_min),  # Canto inferior esquerdo
        (lon_max, lat_min),  # Canto inferior direito
        (lon_max, lat_max),  # Canto superior direito
        (lon_min, lat_max),  # Canto superior esquerdo
        (lon_min, lat_min)   # Fechar o polígono
])
    
    # Calcular o diagrama de Voronoi e limitar ao quadro que contém o Brasil
    pontos = np.vstack(([longitude, latitude], aeroportos_filtrados[['LONGITUDE', 'LATITUDE']].values.astype('float32')))
    region_polys, points = voronoi_regions_from_coords(pontos, square_brasil)

    # Criar o círculo de limite como um polígono
    circle_polygon = gerar_circulo(coordenadas_principais, raio_km)
    mediana_polygon = gerar_circulo(coordenadas_principais, mediana_km)
    decil_6_polygon = gerar_circulo(coordenadas_principais, decil_6_km)
    decil_7_polygon = gerar_circulo(coordenadas_principais, decil_7_km)
    decil_8_polygon = gerar_circulo(coordenadas_principais, decil_8_km)
    decil_9_polygon = gerar_circulo(coordenadas_principais, decil_9_km)

    # Cortar os polígonos de Voronoi pelo circle_polygon (apenas os que estão contidos ou intersectam o círculo)
    voronoi_polygons_clipped = {}
    for key, poly in region_polys.items():
        if not poly.is_empty and isinstance(poly, Polygon):
            # Se o polígono está contido ou intersecta o circle_polygon, usamos apenas a parte dentro do círculo
            if poly.intersects(circle_polygon):
                clipped_poly = poly.intersection(circle_polygon)
                voronoi_polygons_clipped[key] = clipped_poly

    # Identificar o polígono de Voronoi que contém o aeroporto principal
    aeroporto_poly = None
    for poly in voronoi_polygons_clipped.values():
        if not poly.is_empty and isinstance(poly, Polygon):
            # Verifica se o polígono contém as coordenadas principais do aeroporto
            if poly.contains(Point(coordenadas_principais[1], coordenadas_principais[0])):  # Note que o ponto está em (longitude, latitude)
                aeroporto_poly = poly
                break  # Uma vez encontrado, podemos sair do loop
                
    # Criar o mapa (para o aeroporto principal)
    fig = go.Figure()

    if shapefile is not None and aeroporto_poly is not None:
        # Filtrar o shapefile por círculo antes de processar
        shapefile = filtrar_shapefile_por_circulo(shapefile, circle_polygon)

        for _, row in shapefile.iterrows():
            geometry = row['geometry']
            intersection_area_with_aeroporto_poly = geometry.intersection(aeroporto_poly).area
            total_area = geometry.area

            # Regra geral: Verificar interseção com o polígono central
            if intersection_area_with_aeroporto_poly / total_area >= 0.5:
                # Verificar em qual área adicional o município se encontra
                if geometry.intersection(mediana_polygon).area / total_area >= 0.5:
                    fillcolor = 'rgba(255, 165, 0, 0.6)'  # Laranja com opacidade 0.6 (mediana_km)
                    categoria = 'A'
                elif geometry.intersection(decil_6_polygon).area / total_area >= 0.5:
                    fillcolor = 'rgba(255, 165, 0, 0.5)'  # Laranja com opacidade 0.5 (decil_6_km)
                    categoria = 'B'
                elif geometry.intersection(decil_7_polygon).area / total_area >= 0.5:
                    fillcolor = 'rgba(255, 165, 0, 0.4)'  # Laranja com opacidade 0.4 (decil_7_km)
                    categoria = 'C'
                elif geometry.intersection(decil_8_polygon).area / total_area >= 0.5:
                    fillcolor = 'rgba(255, 165, 0, 0.3)'  # Laranja com opacidade 0.3 (decil_8_km)
                    categoria = 'D'
                elif geometry.intersection(decil_9_polygon).area / total_area >= 0.5:
                    fillcolor = 'rgba(255, 165, 0, 0.2)'  # Laranja com opacidade 0.2 (decil_9_km)
                    categoria = 'E'
                else:
                    fillcolor = 'rgba(255, 165, 0, 0.1)'  # Laranja com opacidade 0.1 (fora das áreas definidas)
                    categoria = 'F'
            else:
                # Caso não se encaixe nas regras acima, aplicar as regras de preenchimento azul
                if geometry.intersection(mediana_polygon).area / total_area >= 0.5:
                    fillcolor = 'rgba(0, 0, 255, 0.6)'  # Azul com opacidade 0.6 (mediana_km)
                    categoria = 'G'
                elif geometry.intersection(decil_6_polygon).area / total_area >= 0.5:
                    fillcolor = 'rgba(0, 0, 255, 0.5)'  # Azul com opacidade 0.5 (decil_6_km)
                    categoria = 'H'
                elif geometry.intersection(decil_7_polygon).area / total_area >= 0.5:
                    fillcolor = 'rgba(0, 0, 255, 0.4)'  # Azul com opacidade 0.4 (decil_7_km)
                    categoria = 'I'
                elif geometry.intersection(decil_8_polygon).area / total_area >= 0.5:
                    fillcolor = 'rgba(0, 0, 255, 0.3)'  # Azul com opacidade 0.3 (decil_8_km)
                    categoria = 'J'
                elif geometry.intersection(decil_9_polygon).area / total_area >= 0.5:
                    fillcolor = 'rgba(0, 0, 255, 0.2)'  # Azul com opacidade 0.2 (decil_9_km)
                    categoria = 'K'
                else:
                    fillcolor = 'rgba(0, 0, 255, 0.1)'  # Azul com opacidade 0.1 (fora de todas as áreas definidas)
                    categoria = 'L'
        
            # Armazenar os dados na tabela influencia_municipios com as categorias
            influencia_municipios_data.append({
                'codigo': row['CD_MUN'],
                'nome': row['NM_MUN'],
                'uf': row['SIGLA_UF'],
                'categoria': categoria
            })
            
            # Adicionar o traço correspondente ao município no mapa
            if geometry.geom_type == 'Polygon':
                x, y = geometry.exterior.xy
                fig.add_trace(go.Scattermapbox(
                    lat=list(y),
                    lon=list(x),
                    mode='lines',
                    fill="toself",
                    fillcolor=fillcolor,
                    line=dict(width=0, color='black'),
                    hoverinfo='skip'
                ))
            elif geometry.geom_type == 'MultiPolygon':
                for geom in geometry.geoms:
                    x, y = geom.exterior.xy
                    fig.add_trace(go.Scattermapbox(
                        lat=list(y),
                        lon=list(x),
                        mode='lines',
                        fill="toself",
                        fillcolor=fillcolor,
                        line=dict(width=0, color='black'),
                        hoverinfo='skip'
                    ))
    
    # Adicionar o ponto do aeroporto principal
    fig.add_trace(go.Scattermapbox(
        lat=[latitude],
        lon=[longitude],
        mode='markers',
        marker=go.scattermapbox.Marker(size=14, color='black'),  # Ponto preto para o aeroporto principal
        text=[id_aeroporto_nome],
        hoverinfo='text'
    ))

    # Adicionar os pontos vermelhos para os aeroportos filtrados
    fig.add_trace(go.Scattermapbox(
        lat=aeroportos_proximos_filtrados['LATITUDE'].tolist(),
        lon=aeroportos_proximos_filtrados['LONGITUDE'].tolist(),
        mode='markers',
        marker=go.scattermapbox.Marker(size=10, color='red'),
        text=aeroportos_proximos_filtrados['ID_AEROPORTO'],
        hoverinfo='text'
    ))

    # Adicionar os pontos brancos para os demais aeroportos dentro do círculo
    aeroportos_restantes = aeroportos_proximos.loc[
        ~aeroportos_proximos['ID_AEROPORTO'].isin(aeroportos_filtrados['ID_AEROPORTO'])
    ].copy()
    fig.add_trace(go.Scattermapbox(
        lat=aeroportos_restantes['LATITUDE'].tolist(),
        lon=aeroportos_restantes['LONGITUDE'].tolist(),
        mode='markers',
        marker=go.scattermapbox.Marker(size=10, color='white'),
        text=aeroportos_restantes['ID_AEROPORTO'],
        hoverinfo='text'
    ))

    # Adicionar os pontos azuis para aeroportos_filtrados que não sejam representados como brancos, verdes ou o aeroporto principal
    aeroportos_fora = aeroportos_filtrados.loc[
        (~aeroportos_filtrados['ID_AEROPORTO'].isin(aeroportos_proximos['ID_AEROPORTO'])) &  # Não estão nos aeroportos_proximos
        (aeroportos_filtrados['ID_AEROPORTO'] != id_aeroporto_nome)  # Não é o aeroporto principal (preto)
    ].copy()

    fig.add_trace(go.Scattermapbox(
        lat=aeroportos_fora['LATITUDE'].tolist(),
        lon=aeroportos_fora['LONGITUDE'].tolist(),
        mode='markers',
        marker=go.scattermapbox.Marker(size=10, color='blue'),  # Ponto azul para os aeroportos similares
        text=aeroportos_fora['ID_AEROPORTO'],
        hoverinfo='text'
    ))

    # Ajustar o layout para usar "open-street-map" e configurar o zoom para mostrar o círculo inteiro
    zoom_nivel = calcular_zoom(raio_km)
    fig.update_layout(
        autosize=True,
        margin=dict(l=0, r=0, t=0, b=0),
        showlegend=False,  # Remove a legenda do mapa
        mapbox=dict(
            style="open-street-map",
            center=go.layout.mapbox.Center(
                lat=latitude,
                lon=longitude
            ),
            zoom=zoom_nivel,
        )
    )

    # Retornar o mapa, a tabela de influência dos municípios
    return fig

In [10]:
resultado = id_aeroporto[
    (id_aeroporto['UF'] == 'DF')
]['ID_AEROPORTO'].unique()

resultado

array(['SBBR - BRASÍLIA', 'SIQE - BRASÍLIA'], dtype=object)

In [11]:
piar_aeroportos = ['SBTE - TERESINA', 'SBPB - PARNAÍBA', 'SWKQ - SÃO RAIMUNDO NONATO','SBFZ - FORTALEZA','SNBR - BARREIRAS','SBSL - SÃO LUÍS','SBBR - BRASÍLIA']

In [12]:
for aeroporto in piar_aeroportos:
    # Extrair a sigla do aeroporto (parte antes do primeiro espaço)
    sigla = aeroporto.split(' ')[0].lower()
    
    # Gerar o mapa para cada aeroporto e salvar na variável dinâmica
    globals()[f"mapa_area_influencia_{sigla}"] = gerar_mapa_aeroporto(
        id_aeroporto, aeroporto, ["ALL"], shapefile=municipios, zona_de_influencia=zona_de_influencia
    )
    
    # Salvar o mapa como um arquivo HTML
    globals()[f"mapa_area_influencia_{sigla}"].write_html(f"resultados/mapa_area_influencia_{sigla}.html")